In [15]:
from PIL import Image, ImageDraw
import numpy as np
import os
import pillow_heif

def crop_to_square(image):
    width, height = image.size
    new_size = min(width, height)
    left = (width - new_size) / 2
    top = (height - new_size) / 2
    right = (width + new_size) / 2
    bottom = (height + new_size) / 2
    return image.crop((left, top, right, bottom))

def make_rounded_corners(image, radius_percentage):
    np_image = np.array(image)
    h, w = np_image.shape[:2]
    
    # Calculate radius based on the smaller dimension
    radius = int(min(h, w) * radius_percentage / 100)
    
    # Create a rounded rectangle mask
    mask = Image.new('L', (w, h), 0)
    draw = ImageDraw.Draw(mask)
    draw.rounded_rectangle([(0, 0), (w, h)], radius, fill=255)
    
    # Apply mask to image
    np_mask = np.array(mask)
    np_image = np.dstack((np_image[:, :, :3], np_mask))
    
    return Image.fromarray(np_image, 'RGBA')

def process_images(input_dir, output_dir, radius_percentage=10):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(input_dir):
        if filename.lower().endswith((".png", ".jpg", ".jpeg", ".heif", ".heic", ".tiff", ".bmp")):
            image_path = os.path.join(input_dir, filename)
            try:
                image = Image.open(image_path).convert("RGBA")
            except (IOError, SyntaxError):
                heif_file = pillow_heif.open_heif(image_path)
                image = Image.frombytes(
                    heif_file.mode, 
                    heif_file.size, 
                    heif_file.data, 
                    "raw",
                    heif_file.mode,
                    heif_file.stride,
                ).convert("RGBA")
            
            image = crop_to_square(image)
            image = make_rounded_corners(image, radius_percentage)
            output_path = os.path.join(output_dir, filename.split('.')[0] + ".png")
            image.save(output_path, format="PNG")

input_directory = "cropped_output"
output_directory = "OP"
process_images(input_directory, output_directory, radius_percentage=5)


In [ ]:
from PIL import Image
import os

def resize_image(image, max_width, max_height):
    """Resize an image to fit within the specified dimensions while maintaining aspect ratio."""
    width_ratio = max_width / image.width
    height_ratio = max_height / image.height
    min_ratio = min(width_ratio, height_ratio)

    new_width = int(image.width * min_ratio)
    new_height = int(image.height * min_ratio)

    return image.resize((new_width, new_height), Image.LANCZOS)

def overlay_images(template_path, images_dir, output_dir, position, max_width, max_height):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Get a list of PNG images from the directory
    image_files = [f for f in os.listdir(images_dir) if f.endswith('.png')]

    # Overlay each PNG image onto the template at the specified position
    for image_name in image_files:
        # Open the template image
        template = Image.open(template_path).convert("RGBA")

        # Open the PNG image
        image_path = os.path.join(images_dir, image_name)
        image = Image.open(image_path).convert("RGBA")

        # Resize the image to fit within the specified dimensions
        resized_image = resize_image(image, max_width, max_height)

        # Overlay the resized PNG image onto the template at the specified position
        template.paste(resized_image, position, resized_image)

        # Save the resulting image with the same name as the input image
        output_path = os.path.join(output_dir, image_name)
        template.save(output_path, format='PNG')

        print(f'Saved: {output_path}')

# Set paths and positions
template_path = 'template_padded.jpg'
images_dir = 'OP'
output_dir = 'Final'
position = (310, 350)  # Position where PNGs will be overlaid
max_width, max_height = 500, 500  # Maximum width and height for the resized images

# Run the overlay function
overlay_images(template_path, images_dir, output_dir, position, max_width, max_height)


Saved: Final/241050033_cropped.png
Saved: Final/241040070_cropped.png
Saved: Final/231010408_cropped.png
Saved: Final/241090034_cropped.png
Saved: Final/241020019_cropped.png
Saved: Final/241010022_cropped.png
Saved: Final/241080053_cropped.png
Saved: Final/241050070_cropped.png
Saved: Final/241080027_cropped.png
Saved: Final/241110040_cropped.png


KeyboardInterrupt: 